In [25]:
!pip install mistralai smolagents
!pip install chromadb langchain
!pip install -U langchain
!pip install --upgrade pymupdf

In [26]:
!pip install 'smolagents[litellm]'

In [27]:
import os
import asyncio
from datetime import datetime
from typing import Optional, List, Dict, Any
from smolagents import ToolCallingAgent, tool, CodeAgent, DuckDuckGoSearchTool
from smolagents.models import LiteLLMModel
import pytz
from datetime import datetime, time
import requests
from dotenv import load_dotenv
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from google.colab import userdata
import json

# Загружаем переменные окружения
load_dotenv()

# Настройки Mistral
#key = os.getenv("MISTRAL_API_KEY")

API_KEY = userdata.get('mistral')
MISTRAL_MODEL = "mistral-small-latest"
MISTRAL_BASE_URL = "https://api.mistral.ai/v1"

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
shtraf_dict = {'A123BC124': [{'fine_id': '12345', 'fine_description': 'Нарушение правил парковки'}, {'fine_id': '54321', 'fine_description': 'Нарушение скоростного режима'}],
               'A321BC123': [{'fine_id': '67890', 'fine_description': 'Нарушение правил использования ремня безопасности'}, {'fine_id': '543219', 'fine_description': 'Да'}]}

shtraf_l_dict = {'8503jkfn89': [{'fine_id': '12005', 'fine_description': 'Нарушение правил парковки'}, {'fine_id': '54311', 'fine_description': 'Нарушение скоростного режима'}],
               '56-29vbfdd': [{'fine_id': '61790', 'fine_description': 'Нарушение правил использования ремня безопасности'}, {'fine_id': '545919', 'fine_description': 'Да'}]}

In [30]:
# Путь к папке, где находится chroma.sqlite3
db_path = "/content/drive/MyDrive/Hugging_face/chroma_bd5"
db_path_events = "/content/drive/MyDrive/Hugging_face/chroma_bd_events"

# 1. Подключаемся к существующей БД
client = chromadb.PersistentClient(path=db_path)
client_events = chromadb.PersistentClient(path=db_path_events)

embedding_function = SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"  # Должна совпадать с той, что использовалась при создании
    )

try:
    collection = client.get_collection(
        name="my_documents5",  # Имя вашей коллекции
        embedding_function=embedding_function
    )
    print(f"✅ Коллекция 'my_documents' загружена")
    print(f"✅ Документов в коллекции: {collection.count()}")

except Exception as e:
    print(f"❌ Ошибка при загрузке коллекции: {e}")
    print("\nПопробуйте:")
    print("1. Проверить имя коллекции в client.list_collections()")
    print("2. Убедиться, что модель эмбеддингов совпадает")


try:
    collection_events = client_events.get_collection(
        name="my_documents_events",  # Имя вашей коллекции
        embedding_function=embedding_function
    )
    print(f"✅ Коллекция 'my_documents_events' загружена")
    print(f"✅ Документов в коллекции: {collection_events.count()}")

except Exception as e:
    print(f"❌ Ошибка при загрузке коллекции: {e}")
    print("\nПопробуйте:")
    print("1. Проверить имя коллекции в client_events.list_collections()")
    print("2. Убедиться, что модель эмбеддингов совпадает")

✅ Коллекция 'my_documents' загружена
✅ Документов в коллекции: 41
✅ Коллекция 'my_documents_events' загружена
✅ Документов в коллекции: 62


In [32]:
@tool
def get_shtraf(car_number: str = None, driver_license: str = None, UIN: str = None) -> str:
    """
    Получить штраф по номеру автомобиля, номеру водительского удостоверения или УИН.

    Args:
        car_number: номер автомобиля (например, 'А100ВС124')
        driver_license: номер водительского удостоверения (например, '9910 100001')
        UIN: УИН (например, '18810125000000000002')

    Returns:
        Строка с описанием штрафов, если они есть, или сообщение 'Штрафов не найдено'
    """

    shtrafs = shtraf_dict.get('car_number', 'Штрафов на автомобиле нет') #+ shtraf_l_dict.get('driver_license', 'Штрафов у водителя нет')

    return shtrafs

@tool
def fine_paying(fine_id: List[Dict[str, str]], card_number: str) -> str:
  """
    Функция для оплаты штрафа.

    Args:
        fine_id: Список уникальных идетификаторов штрафов (например, ["65927", "12309"])
        card_number: номер карты, с которой будет производиться оплата (например, "2022 9527 3301 5855")

    Returns:
        Информация о успешности или неуспешности оплаты
    """

  return 'Оплата прошла'

@tool
def call_doctor(address: str, insurance: str, phone_number: str) -> str:
    """
    Функция для вызова врача на дом.

    Args:
        address: Адрес, по которому требуется вызвать врача (например, г.Красноярск, улица Ленина, дом 24)
        insurance: Номер страховки(например, 123)
        phone_number: Номер телефона обратившегося человека (например, +79779934385)

    Returns:
        Информация о том, вызван врач или нет
    """

    # Устанавливаем временную зону (можно изменить на нужную)
    timezone = pytz.timezone('Asia/Krasnoyarsk')  # Для Красноярска
    now = datetime.now(timezone)

    # Определяем рабочее время (с 8:00 до 18:00)
    work_start = time(8, 0, 0)
    work_end = time(18, 0, 0)
    current_time = now.time()

    # Проверяем, рабочее ли сейчас время
    if not (work_start <= current_time <= work_end):
        return (f'В вызове врача отказано: сейчас нерабочее время.\n'
                f'Текущее время: {current_time.strftime("%H:%M")}\n'
                f'Врачи работают с {work_start.strftime("%H:%M")} до {work_end.strftime("%H:%M")}\n'
                f'Врач приедет завтра с 8:00.')

    # Если рабочее время, проверяем сколько времени осталось до конца рабочего дня
    end_of_day = datetime.combine(now.date(), work_end)
    time_until_end = (end_of_day - now).total_seconds() / 3600  # в часах

    if time_until_end < 1:
        # До конца рабочего дня меньше часа
        return (f'Врач вызван по адресу: {address}\n'
                f'Так как до конца рабочего дня осталось менее часа, врач приедет завтра с 8:00.\n'
                f'Текущее время: {current_time.strftime("%H:%M")}')
    else:
        # Достаточно времени, врач приедет в течение часа
        arrival_time = now.replace(minute=0, second=0, microsecond=0)
        arrival_time = arrival_time.replace(hour=arrival_time.hour + 1)

        # Если следующий час выходит за рамки рабочего дня
        if arrival_time.time() > work_end:
            return (f'Врач вызван по адресу: {address}\n'
                    f'Врач приедет завтра с 8:00.\n'
                    f'Текущее время: {current_time.strftime("%H:%M")}')

        return (f'Врач вызван по адресу: {address}\n'
                f'Примерное время приезда врача: {arrival_time.strftime("%H:%M")}\n'
                f'Текущее время: {current_time.strftime("%H:%M")}')

@tool
def check_adress(adress: str) -> str:
    """
    Функция для для проверки существования адреса.

    Args:
        adress: Адрес, по которому требуется вызвать врача (например, г.Красноярск, улица Ленина, дом 24)

    Returns:
        Информация о существовании такого адреса
    """

    return 'Такой адрес существует'

@tool
def check_insurance(insurance: str) -> str:
    """
    Функция для для проверки полиса.

    Args:
        insurance: Номер страховки(например, 123)

    Returns:
        Информация о существовании такого полиса
    """

    return 'Такой полис существует'

@tool
def check_phone_number(phone_number: str) -> str:
    """
    Функция для проверки номера телефона на корректность.

    Args:
        phone_number: Номер телефона обратившегося человека (например, +79779934385)

    Returns:
        Информация о корректности номера телефона
    """

    return 'Номер телефона корректен'

@tool
def get_current_time(timezone: str = "Europe/Moscow") -> str:
    """
    Получить текущее время в указанном часовом поясе.

    Args:
        timezone: Часовой пояс в формате IANA (например, 'Europe/Moscow', 'America/New_York')

    Returns:
        Строка с текущей датой и временем
    """
    try:
        tz = pytz.timezone(timezone)
        current_time = datetime.now(tz)

        # Русские названия месяцев
        months_ru = {
            1: 'января', 2: 'февраля', 3: 'марта', 4: 'апреля',
            5: 'мая', 6: 'июня', 7: 'июля', 8: 'августа',
            9: 'сентября', 10: 'октября', 11: 'ноября', 12: 'декабря'
        }

        # Русские названия дней недели
        weekdays_ru = {
            'Monday': 'понедельник',
            'Tuesday': 'вторник',
            'Wednesday': 'среда',
            'Thursday': 'четверг',
            'Friday': 'пятница',
            'Saturday': 'суббота',
            'Sunday': 'воскресенье'
        }

        weekday_en = current_time.strftime("%A")
        weekday_ru = weekdays_ru.get(weekday_en, weekday_en)

        date_str = f"{current_time.day} {months_ru[current_time.month]} {current_time.year} года"
        time_str = current_time.strftime("%H:%M:%S")

        return (f"Текущее время в часовом поясе {timezone}:\n"
                f"📅 {date_str}\n"
                f"🕐 {time_str}\n"
                f"📆 {weekday_ru}")

    except pytz.exceptions.UnknownTimeZoneError:
        return f"❌ Часовой пояс '{timezone}' не найден. Используйте корректный часовой пояс (например, 'Europe/Moscow')."
    except Exception as e:
        return f"❌ Ошибка: {str(e)}"

@tool
def get_current_time() -> str:
    """
    Получить текущее время и дату.

    Returns:
        Текущая дата и время в формате 'ДД.ММ.ГГГГ ЧЧ:ММ'
    """
    # Можно использовать локальное время или указать конкретную временную зону
    timezone = pytz.timezone('Asia/Krasnoyarsk')  # Для Красноярска
    now = datetime.now(timezone)
    return f"Текущее время: {now.strftime('%d.%m.%Y %H:%M')}"

@tool
def get_doctor_working_hours() -> str:
    """
    Получить информацию о рабочем времени врачей.

    Returns:
        Расписание работы врачей
    """
    return "Врачи работают ежедневно с 8:00 до 18:00."

@tool
def calculate_doctor_arrival(address: str) -> str:
    """
    Рассчитать возможное время прибытия врача на указанный адрес.

    Args:
        address: Адрес пациента

    Returns:
        Информация о возможности и времени прибытия врача
    """

    return 'Приезд врача займёт 1 час'

@tool
def search_privilege_documents(query: str, n_results: int = 10) -> List[Dict[str, Any]]:
    """
    Ищет релевантные документы в векторной базе данных по текстовому запросу.

    Args:
        query: Поисковый запрос пользователя
        n_results: Количество возвращаемых документов (по умолчанию 3)

    Returns:
        Список словарей с найденными документами и их метаданными
    """
    try:
        # Выполняем поиск (используем вашу существующую функцию)
        results = collection.query(
            query_texts=[query],
            n_results=n_results
        )

        # Форматируем результаты для удобства использования агентом
        formatted_results = []
        for i in range(len(results['ids'][0])):
            formatted_results.append({
                'id': results['ids'][0][i],
                'document': results['documents'][0][i],
                'metadata': results['metadatas'][0][i],
                'distance': float(results['distances'][0][i]),  # Конвертируем в float для сериализации
                'relevance_rank': i + 1
            })

        return formatted_results

    except Exception as e:
        return [{'error': f"Ошибка при поиске: {str(e)}"}]

@tool
def search_events(query: str, n_results: int = 10) -> List[Dict[str, Any]]:
    """
    Ищет релевантные мероприятия в векторной базе данных по текстовому запросу.

    Args:
        query: Поисковый запрос, сформированный помощником (это ты)
        n_results: Количество возвращаемых документов (по умолчанию 10)

    Returns:
        Список словарей с найденными документами и их метаданными
    """
    try:
        # Выполняем поиск (используем вашу существующую функцию)
        results = collection_events.query(
            query_texts=[query],
            n_results=n_results
        )

        # Форматируем результаты для удобства использования агентом
        formatted_results = []
        for i in range(len(results['ids'][0])):
            formatted_results.append({
                'id': results['ids'][0][i],
                'document': results['documents'][0][i],
                'metadata': results['metadatas'][0][i],
                'distance': float(results['distances'][0][i]),  # Конвертируем в float для сериализации
                'relevance_rank': i + 1
            })

        return formatted_results

    except Exception as e:
        return [{'error': f"Ошибка при поиске: {str(e)}"}]

@tool
def event_enrolling(name: str, stud_number: str) -> str:
    """
    Производит запись на мероприятие.

    Args:
        name: Полное имя пользователя (например, Янук Андрей Владимирович)
        stud_number: Номер студенческого билета (например, 123)

    Returns:
        Информация об успешности или неуспешности записи на мероприятии
    """

    if len(stud_number) > 2 and name.split() >= 2:
        return 'Запись осуществлена'
    else:
        return 'Запись на мероприятие отклонена'

@tool
def get_fine_prompt() -> str:
    """
    Возвращает инструкцию к действиям, когда пользователь интересуется штрафами.

    Returns:
        Промпт (инструкцию) для случая, когда пользователь задал вопрос о штрафах.
    """
    with open(r'/content/drive/MyDrive/Hugging_face/fine_prompt.txt', encoding='utf-8') as file:
        prompt_text = file.read()

    return prompt_text

@tool
def get_events_prompt() -> str:
    """
    Возвращает инструкцию к действиям, когда пользователь интересуется штрафами.

    Returns:
        Промпт (инструкцию) для случая, когда пользователь задал вопрос о штрафах.
    """
    with open(r'/content/drive/MyDrive/Hugging_face/events_prompt.txt', encoding='utf-8') as file:
        prompt_text = file.read()

    return prompt_text

In [ ]:
class MaxAgent:
    """ИИ-агент с доступом к инструментам получения штрафов и вызова врача через Mistral API"""

    def __init__(self, api_key: Optional[str] = None, model: str = None):
        """
        Инициализация агента.
        """
        self.api_key = api_key or API_KEY
        self.model = model or MISTRAL_MODEL

        if not self.api_key:
            raise ValueError(
                "Не указан API ключ Mistral. "
                "Установите MISTRAL_API_KEY в переменных окружения."
            )

        # Собираем инструменты
        self.tools = [
            get_fine_prompt,
            get_events_prompt,
            get_shtraf,
            fine_paying,
            call_doctor,
            check_adress,
            check_insurance,
            check_phone_number,
            get_current_time,
            get_doctor_working_hours,
            calculate_doctor_arrival,
            search_privilege_documents,
            search_events,
            event_enrolling,
        ]

        # Системный промпт
        self.system_prompt = self._create_system_prompt()

        try:
            # Импортируем LiteLLMModel здесь, чтобы не было ошибки импорта
            from smolagents.models import LiteLLMModel

            # Настраиваем модель Mistral через LiteLLMModel
            model_config = {
                "model": f"mistral/{self.model}",
                "api_key": self.api_key,
                "temperature": 0.1,
                "max_tokens": 1000,
            }

            self.llm = LiteLLMModel(**model_config)

        except Exception as e:
            print(f"⚠️  Не удалось использовать Mistral: {e}")

        # Создаем агента с системным промптом
        self.agent = ToolCallingAgent(
            model=self.llm,
            tools=self.tools,
            max_steps=10,
            add_base_tools=False,
            verbosity_level=1,
        )
        #print(dir(self.agent.system_prompt))

        self.agent.prompt_templates["system_prompt"] = self.system_prompt

    def _create_system_prompt(self) -> str:
        """Создать системный промпт"""
        return  """
                Ты - помошник, который может подсказать информацию о наличии штрафов, проконсультировать по поводу полагающихся пользователю льгот, подсказать интересные пользователю мероприятия в его ВУЗе и подобрать пользователю университет для поступления.
                Твоя первая задача, определить, что хочет пользователь:
                - получить информацию о штрафах
                - получить информацию о льготах
                - получить информацию о мероприятиях в ВУЗе
                - подобрать ВУЗ для поступления.

                После определения желания пользователя вызови одну из следующих функций:
                - вызови функцию get_fine_prompt(), которая возвращает дальнейшую инструкцию для действий в случае, когда пользователя интересуют штрафы
                - вызови функцию get_event_prompt(), которая возвращает дальнейшую инструкцию для действий в случае, когда пользователя интересуют мероприятиями в его ВУЗе
                - вызови функцию get_privilege_prompt(), которая возвращает дальнейшую инструкцию для действий в случае, когда пользователя интересуют льготы
                - вызови функцию get_university_prompt(), которая возвращает дальнейшую инструкцию для действий в случае, когда пользователя интересует ВУЗ для поступления и учёбы

                Доступные функции:
                Доступные инструменты:
                1. get_shtraf(type, type_id) - получить штраф по номеру автомобиля, номеру водительского удостоверения или УИН.
                2. call_doctor(adress, insurance, phone_number) - вызвать врача на дом
                3. check_adress(adress) - проверить существование адреса
                4. check_insurance(insurance) - проверить существования полиса
                5. check_phone_number(phone_number) - проверить корректности номера телефона
                6. get_current_time() - получить текущее время
                7. get_doctor_working_hours() - получить время работы врача
                8. calculate_doctor_arrival() - получить время, необходимое для прибытия врача
                9. search_privilege_documents(query, n_results) - получение информации о льготах из базы данных
                10. fine_paying(fine_id, card_number) - Функция для оплаты штрафов.
                11. search_privilege_documents() - Поиск релевантных документов в базе данных со льготами
                12. search_ events() - Поиск релевантных документов в базе данных с мероприятиями в ВУЗах
                """

    def ask(self, question: str) -> str:
        """Задать вопрос агенту"""
        try:
            # Создаем промпт с системным сообщением и вопросом пользователя
            #full_prompt = f"{self.system_prompt}\n\nВопрос пользователя: {question}"
            full_prompt = f"Вопрос пользователя: {question}"

            # Запускаем агента
            response = asyncio.run(self.agent.run(full_prompt))
            return response
        except Exception as e:
            return f"❌ Ошибка: {str(e)}"

    def interactive_chat(self):
        """Интерактивный чат"""
        print("=" * 60)
        print("🤖 Ассистент по времени")
        print(f"📡 Модель: {self.model}")
        print("💬 Введите 'выход' для завершения")
        print("=" * 60)

        while True:
            try:
                user_input = input("\n👤 Вы: ").strip()

                if user_input.lower() in ['выход', 'exit', 'quit', 'q']:
                    print("\n👋 До свидания!")
                    break

                if not user_input:
                    continue

                print("\n🤔 Думаю...")
                response = self.ask(user_input)
                print(f"🤖 Ассистент: {response}")

            except KeyboardInterrupt:
                print("\n\n🛑 Завершение работы...")
                break
            except Exception as e:
                print(f"\n❌ Ошибка: {e}")

In [34]:
def main():
    """Главная функция"""
    print("🚀 Запуск ассистента\n")

    # Получаем API ключ
    api_key = API_KEY


    selected_model = 'mistral-small-latest'

    print(f"\n🎯 Использую модель: {selected_model}")

    try:
        # Создаем агента
        print("\n🎭 Создаю агента...")
        agent = MaxAgent(api_key=api_key, model=selected_model)

        # Показываем возможности
        print("\n✨ Готов к работе!")
        print("\n" + "=" * 50)

        # Запускаем чат
        agent.interactive_chat()

    except Exception as e:
        print(f"\n❌ Ошибка: {e}")
        import traceback
        traceback.print_exc()

In [36]:
main()

/usr/local/lib/python3.12/dist-packages/smolagents/models.py:1196: FutureWarning: The 'model_id' parameter will be required in version 2.0.0. Please update your code to pass this parameter to avoid future errors. For now, it defaults to 'anthropic/claude-3-5-sonnet-20240620'.
  warnings.warn(


🚀 Запуск ассистента


🎯 Использую модель: mistral-small-latest

🎭 Создаю агента...

✨ Готов к работе!

🤖 Ассистент по времени
📡 Модель: mistral-small-latest
💬 Введите 'выход' для завершения

👤 Вы: Хочу узнать о мероприятиях в моём университете

🤔 Думаю...


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Вопрос пользователя: Хочу узнать о мероприятиях в моём университете                                             │
│                                                                                                                 │
╰─ LiteLLMModel - anthropic/claude-3-5-sonnet-20240620 ───────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_events_prompt' with arguments: {}                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Вот список мероприятий, которые могут быть вам         │
│ интересны в вашем университете (МГУ им. Ломоносова) по тематике программирования и информационных технологий.   │
│ Вы можете записаться на одно из них. Напишите мне название мероприятия, на которое вы хотите записаться, и я    │
│ помогу вам с записью на него.'}                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Вот список мероприятий, которые могут быть вам интересны в вашем университете (МГУ им. Ломоносова) по
тематике программирования и информационных технологий. Вы можете записаться на одно из них. Напишите мне название 
мероприятия, на которое вы хотите записаться, и я помогу вам с записью на него.

Final answer: Вот список мероприятий, которые могут быть вам интересны в вашем университете (МГУ им. Ломоносова) по
тематике программирования и информационных технологий. Вы можете записаться на одно из них. Напишите мне название 
мероприятия, на которое вы хотите записаться, и я помогу вам с записью на него.

[Step 4: Duration 1.37 seconds| Input tokens: 13,602 | Output tokens: 140]

🤖 Ассистент: ❌ Ошибка: asyncio.run() cannot be called from a running event loop

👤 Вы: Выход

👋 До свидания!


In [ ]:
#Хочу вызвать врача на дом. Адрес: Красноярск, л.Плеханова, д. 27, полис - 123, номер телефона +79779934385